In [27]:
import mysql.connector
import pandas as pd
from news_scraping import scraping
from nltk.corpus import stopwords
import gensim
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from fuzzywuzzy import process
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
scraping()

Iniciando scraping do site G1. Isto pode demorar um pouco...
Buscando notícias e salvando em lista...
Iniciando scraping do site Google News...
Buscando notícias e salvando em lista...
Iniciando scraping do site Estadão...
Buscando notícias e salvando em lista...
Importando listas para o banco de dados...


In [28]:
db = mysql.connector.connect(
   host="database-1.cpqjjsrzpykc.us-east-2.rds.amazonaws.com",
   user="admin",
   passwd="Andre.2021",
   database="noticias"
)

print(db)

In [29]:
cursor = db.cursor()
query = ("SELECT * FROM noticias.news")
cursor.execute(query)
records = cursor.fetchall()
db.close()

In [30]:
records[0]

(1,
 'Alison e Álvaro perdem, e Brasil fica sem medalha no vôlei de praia pela 1ª vez',
 '',
 'Há 2 horas',
 'https://ge.globo.com/olimpiadas/noticia/alison-e-alvaro-perdem-nas-quartas-de-final-do-volei-de-praia.ghtml',
 'https://s2.glbimg.com/x85VfdtXMrfxpMDqclxBVoRhIKg=/0x108:3793x2242/540x304/smart/filters:max_age(3600)/https://i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2021/2/Q/B6wpyASrAyokcjZHpGMQ/2021-08-04t012627z-664316840-sp1eh840401n3-rtrmadp-3-olympics-2020-vbv-m-team2-qfnl-000200.jpg',
 'G1')

In [31]:
news_df = pd.DataFrame(records, columns=['id','titulo','subtitulo','hora','link','img', 'fonte'])
news_df = news_df.drop(columns=['id'])

In [32]:
news_df['titulo'].isnull().sum()

0

In [33]:
news_df.shape

(114, 6)

In [34]:
def process_texto(s):
    
    # seleciona apenas letras (lembrando que o texto está em português e as letras possuem acento)
    texto_limpo = gensim.utils.simple_preprocess(s)
    
    # remove stopwords
    sw = stopwords.words('portuguese')
    sem_stopwords = [word for word in texto_limpo if word not in sw]
    
    return sem_stopwords

In [35]:
news_df['titulo_sw'] = news_df['titulo'].apply(lambda s: process_texto(s)).apply(lambda x: ' '.join(x))

In [36]:
news_df.head(2)

,titulo,subtitulo,hora,link,img,fonte,titulo_sw
0,"Alison e Álvaro perdem, e Brasil fica sem meda...",,Há 2 horas,https://ge.globo.com/olimpiadas/noticia/alison...,https://s2.glbimg.com/x85VfdtXMrfxpMDqclxBVoRh...,G1,alison álvaro perdem brasil fica medalha vôlei...
1,ASSISTA: Dora Varella e Yndiara Asp avançam no...,,Há 1 hora,https://ge.globo.com/olimpiadas/ao-vivo/olimpi...,https://s2.glbimg.com/XqpZlXZP1s_AlVY6QKKEAwDN...,G1,assista dora varella yndiara asp avançam skate...


In [37]:
# Bag-of-Words (bow) vectorises the data
# transforming tokens into useful features
     
bow_transformer = CountVectorizer().fit_transform(news_df['titulo_sw'])

news_tfidf = TfidfTransformer().fit_transform(bow_transformer)


In [38]:
nn = NearestNeighbors(metric='cosine', n_neighbors=4)

In [39]:
# Recomendador de notícias

lista = []
def recommender(noticia):
    lista.clear()
    nn.fit(news_tfidf)
    idx=process.extractOne(noticia, news_df['titulo'])[2]
    print('Notícias recomendadas: ')
    distances, indices=nn.kneighbors(news_tfidf[idx], n_neighbors=4)
    n=0
    for i in indices:
        lista.append(i)
        
    for n in lista[0][1:]:
        print(f'{news_df.iloc[n]} \n\n')
        

In [40]:
recommender(news_df['titulo'][1])

Notícias recomendadas: 
titulo       Olimpíada de Tóquio 2021: Brasil retoma disput...
subtitulo                                                     
hora                                             6 horas atrás
link         news.google.com/articles/CBMiLWh0dHBzOi8vd3d3L...
img          https://lh3.googleusercontent.com/proxy/dYbBAD...
fonte                                               BBC Brasil
titulo_sw    olimpíada tóquio brasil retoma disputas skate ...
Name: 99, dtype: object 


titulo       Medalhista de novo: assista ao salto que deu o...
subtitulo                                                     
hora                                               Há 16 horas
link         https://g1.globo.com/mundo/video/thiago-braz-v...
img                                                           
fonte                                                       G1
titulo_sw    medalhista novo assista salto deu bronze thiag...
Name: 17, dtype: object 


titulo       Cachorro 'fã do Jornal Naci